# End-To-End Training Example of RNN for rain estimation and detection using PyNNCML
This notebook presents an end-to-end example of training a Recurrent Neural Network (RNN) based rain estimation neural network [1] on the openMRG dataset.
This tutorial is built using the PyNNCML software package, which provides both tools to simplify the training process of deep learning models for CMLs.
We start with obtaining a subset of the OpenMRG dataset and splitting it into training and validation datasets.
Afterward, we construct the One Step Network from [1], followed by the training loop.
Finally, we analyze the model performance in terms of RMSE and Bias.


Notebook structure
1. Imports and Installation of PyNNCML
2. Hyperparameter settings
3. Build Dataset
4. Build Neural Network
5. Training Loop
6. Neural Network Analysis


[Run this notebook in colab](https://colab.research.google.com/github/haihabi/PyNNcml/blob/master/examples/training_rnn.ipynb)

To run this notebook on Colab using GPU, please do the following:
Runtime -> Change runtime type -> Select GPU -> Save


In [ ]:
import sys
import os

if os.path.exists('../../pynncml'):
    print("Import PyNNCML From Code")
    sys.path.append('../../')  # This line is need to import pynncml
else:
    print("Install PyNNCML From pip")
    !pip install pynncml

import pynncml as pnc
import torch
from tqdm import tqdm
from matplotlib import pyplot as plt
import wandb
import random
import numpy as np
from examples.rain_score.score.score_matching_loss import ScoreMatchingLoss

np.random.seed(0)
torch.manual_seed(0)
random.seed(0)



In [ ]:
batch_size = 128
window_size = 32
lr = 1e-4
n_epochs = 3200

xy_min = None
xy_max = None
time_slice = slice("2015-06-01", "2015-08-14")  # Time Interval
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Building Training and Validation datasets

In [ ]:
dataset_base = pnc.datasets.loader_open_mrg_dataset(xy_min=xy_min, xy_max=xy_max, time_slice=time_slice)
dataset = pnc.datasets.linkdataset2subsequent(dataset_base)
dataset_base.link_set.plot_links()
plt.grid()
plt.tick_params(left=False, right=False, labelleft=False, labelbottom=False, bottom=False)
plt.show()

training_dataset, validation_dataset = torch.utils.data.random_split(dataset, [0.8, 0.2])
data_loader_all = torch.utils.data.DataLoader(dataset_base, batch_size)
data_loader = torch.utils.data.DataLoader(training_dataset, batch_size)
val_loader = torch.utils.data.DataLoader(validation_dataset, batch_size)


# Loss


In [ ]:
loss_function = ScoreMatchingLoss(torch.ones(1) * 0.1, torch.inf * torch.ones(1)).to(device)


In [ ]:
rain_data = np.asarray(dataset.label)
from scipy.stats import expon, gamma

res = expon.fit(rain_data)
res_gamma = gamma.fit(rain_data - 0.1)
print(res, res_gamma, )
min_r, max_r, mean_r = np.min(rain_data), np.max(rain_data), np.mean(rain_data)
r_array = np.linspace(min_r, max_r)
plt.hist(rain_data, density=True, bins=50, label="Histogram")
plt.plot(r_array, expon.pdf(r_array, loc=res[0], scale=res[1]), label="PDF")
plt.plot(r_array, gamma.pdf(r_array, a=res_gamma[0], loc=0, scale=res_gamma[2]), label="PDF Gamma")
plt.grid()
plt.legend()
plt.xlabel("Rain-Rate[mm/hour]")
plt.ylabel("Probability")
plt.show()



# Training Loop

In [ ]:
from examples.rain_score.score.ema import ModelEma
from examples.rain_score.conformer.conditional_encoder import RainScoreConformer

model = RainScoreConformer(normalization_cfg=pnc.training_helpers.compute_data_normalization(data_loader_all)).to(
    device)
ema = ModelEma(model)
min_value = torch.ones(1).to(device) * 0.1
max_value = torch.inf * torch.ones(1).to(device)
lr = 1e-6
warmup_epochs = 50
n_epochs = 4000
div_factor = 100
wandb.init(project="rain_score")
model.update_dropout(0.9)
opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0.0)
scheduler_cosin = torch.optim.lr_scheduler.CosineAnnealingLR(opt, n_epochs - warmup_epochs,
                                                             eta_min=lr / div_factor)
scheduler_warmup = torch.optim.lr_scheduler.LinearLR(opt, 1 / div_factor, 1, total_iters=warmup_epochs)
scheduler = torch.optim.lr_scheduler.SequentialLR(opt, [scheduler_warmup, scheduler_cosin], [warmup_epochs])

ra = pnc.analysis.ResultsAccumulator()
am = pnc.analysis.AverageMetric()
model.train()
print("Start Training")
pbar = tqdm(range(n_epochs), desc='description')
pass_loss_sm = None
pass_loss_reg = None
for epoch in pbar:
    am.clear()
    model.train()
    prob = 0.9 - 0.4 * (epoch + 1) / n_epochs
    model.update_dropout(prob)
    for rain_rate, data, metadata in data_loader:
        opt.zero_grad()

        data = data.to(device)
        # Random Slice
        slice = int(np.random.randint(8, data.shape[1], 1)[0])
        data = data[:, -slice:, :]
        # Dequantization noise
        data = data + torch.rand_like(data) * 0.3 - 0.15  # Dequantization noise
        # Baseline Shift
        baseline_shift = torch.permute(
            torch.nn.functional.avg_pool1d(
                torch.permute(torch.randn(data.shape)[:, :, :int(data.shape[-1] / 2)], [0, 2, 1]), 9), [0, 2, 1])
        data[:, :, :int(data.shape[-1] / 2)] += baseline_shift
        data[:, :, int(data.shape[-1] / 2):] += baseline_shift

        metadata = metadata.to(device)
        rain_rate = rain_rate.reshape([-1, 1]).float().to(device)
        noise = 0
        _rr = torch.tensor(rain_rate.data + noise, device=device,
                           requires_grad=True)

        rain_hat, prior_out = model(data, metadata.to(device), _rr)

        rain_hat_sm, rain_hat_reg = rain_hat.clone(), rain_hat.clone()

        loss_sm = loss_function(rain_hat_sm, _rr)
        loss = loss_sm
        loss.backward()
        opt.step()

        pass_loss_sm = loss_sm.item()
        ema.update(model)

        am.add_results(loss=loss.item(), loss_sm=loss_sm.item())

    scheduler.step()

    loss_avg = am.get_results("loss")
    pbar.set_description(f"Loss Value {loss_avg}")
    res = {"loss": am.get_results("loss"),
           "loss_sm": am.get_results("loss_sm")}
    wandb.log(res)
    ra.add_results(**res)
wandb.finish()

Loss Value 2.3202218410157007:   1%|          | 49/4000 [39:23<35:40:04, 32.50s/it] C:\Users\haiha\.conda\envs\Research\lib\site-packages\torch\optim\lr_scheduler.py:152: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)
Loss Value -504.71315877367283:  17%|█▋        | 698/4000 [3:14:08<13:06:03, 14.28s/it] 

In [ ]:
torch.save(model.state_dict(), "model_score.pkl")
torch.save(ema.state_dict(), "model_score_ema.pkl")